# Multi .csv data reader and starter to perform tasks
First, zero-crossing data were preprocessed using `readAnabat()` from AnabatTools.R in R. The preprocessing returns .csv files that contains frequency (kHz) and time (microsecond).

In [1]:
import os
import glob
import pandas as pd

In [5]:
#loop .csv files to perform any task
#example return number of row

for filenames in glob.glob('../data/Multi_data_Trialsize/*.csv'):
    reader=pd.read_csv(filenames)
    nrow= reader.shape [0]
    print nrow

6208
12692
6058


In [6]:
#list of .csv files in the working directory
nfile=[]
for filenames in glob.glob('../data/Multi_data_Trialsize/*.csv'):
    nfile.append(filenames)

nfile

['../data/Multi_data_Trialsize\\P7132033_37.csv',
 '../data/Multi_data_Trialsize\\P7132035_14.csv',
 '../data/Multi_data_Trialsize\\P7132037_05.csv']

In [7]:
#example reading first nfile
print pd.read_csv(nfile[0]).head()

      Filename  Time  Frequency
0  P7132033_37    10        NaN
1  P7132033_37    99        NaN
2  P7132033_37   183    46242.0
3  P7132033_37   270    46783.0
4  P7132033_37   359    45454.0


# Processing zero-crossing files with bat.py
bat.py is in-house tools to extract zero crossing modified from ZCANT `extract_anabat()` function and to remove noise.

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import pandas as pd
import glob
import errno
from util.bat import *

# Set some Pandas options
pd.set_option('notebook_repr_html', True)
pd.set_option('max_columns', 30)
pd.set_option('max_rows', 20)

In [20]:

def get_batcall_csv(datadir,outdir):
    """
        Given a folder directory extract zero crossing data and create .csv files
    """
    
    filenames = glob.glob(datadir + '/*#')

    info=list()
    

    for filename in glob.glob(datadir + '/*#'):
        signal=list(extract_anabat(filename))
        signal.append(filename)
        info.append(signal)
    
    for i, item in enumerate (filenames):
        t=info[i][0]
        freq=info[i][1]
        ampl=info[i][2] #Empty array
        metadata=info[i][3]
        filename=np.repeat(info[i][4][-12:],len(t))
    
        columns=["Filename","Time", "Frequency"] #,"Label0","Label1","Label2"]
        batcall_df=pd.DataFrame({'Filename':filename,'Time':t,'Frequency':freq}, columns=columns) #,'Label0':label0,'Label1':label1,'Label2':label2}, columns=columns)
        batcall_df.to_csv(outdir + '/' + info[i][4][-12:] + ".csv", index=False)
    

In [4]:
os.getcwd()   

'C:\\Users\\bty\\Desktop\\CSC505\\Bat_Echolocation\\src'

In [10]:
os.chdir('C:/Users/bty/Desktop/CSC505/Bat_Echolocation/data')

In [22]:
#Create outdir in current directory

path='./outfun'
try:
    os.mkdir(path)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [23]:
datadir='C:/Users/bty/Desktop/CSC505/Bat_Echolocation/data/social'
outdir='C:/Users/bty/Desktop/CSC505/Bat_Echolocation/data/outfun'
get_batcall_csv(datadir,outdir)

In [24]:
nfile=[]
for filenames in glob.glob('../data/outfun/*.csv'):
    nfile.append(filenames)

nfile

['../data/outfun\\S8072135.07#.csv',
 '../data/outfun\\S8072143.12#.csv',
 '../data/outfun\\S8072159.22#.csv']

In [25]:
print pd.read_csv(nfile[0]).head()

       Filename      Time     Frequency
0  S8072135.07#  0.000108  38095.238095
1  S8072135.07#  0.000211  37383.177570
2  S8072135.07#  0.000318  37735.849057
3  S8072135.07#  0.000425  37383.177570
4  S8072135.07#  0.000530  37383.177570
